# Feature Selection with Genetic Algorithm and Random Forest on Genotyped Data for the Pharmacogenetic Panel

In this Jupyter notebook, we present the execution of a genetic algorithm for feature selection (feature selection) based on a custom implementation. The objective is to obtain, for a set of good, bad responders or controls in the One-Hot-Encoding (present or not present mutation) format, which are the SNPs most representative for that classification. Once obtained these SNPs, the variables less interesting are eliminated and a supervised or unsupervised machine learning analysis is performed.

## Load of libraries and required modules

In [1]:
import os
import time
import random
import multiprocessing as mp
from functools import partial

import pandas as pd

from genetic_algorthim import GeneticAlgorithm
from data_processing import splitDataByColumn

## Matrix loading in One-Hot encoding format

This matrix contains the genotype data for several SNPs for the study samples together with their respective category (good or bad responders to a drug and controls). We load the dataset, store separately the information of the samples and their respective class and the SNP data in One-Hot-Encoding format.

In [2]:
data_path = '/Users/alzorrcarri/projects_genyo/ml_farmacogen/data/nas_imputed_joined.tsv'
data = pd.read_csv(data_path, sep='\t', header=0)
data.head()

,rs2493297,rs2493292,rs2493291,rs3820586,rs3795713,rs4342887,rs1140279,rs645128,rs659659,rs2359242,...,rs6060048,rs7520,rs1789953,rs1788466,rs12774,rs1047209,rs2839173,rs2839174,sample_id,class
0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,1/1,1/1,1/1,...,0/0,1/1,0/1,0/1,1/1,0/0,0/0,0/0,CaP-18-001-MJ-PI-T17-A3,Good
1,0/1,0/0,0/0,0/1,0/1,0/1,0/1,1/1,0/1,0/1,...,0/1,1/1,0/0,0/0,0/1,0/1,0/1,0/1,CaP-18-002-MJ-PA-T2-A9,Good
2,0/0,0/0,0/0,0/0,0/0,0/0,0/0,1/1,1/1,1/1,...,0/0,0/0,0/0,0/1,0/1,0/0,0/1,0/1,CaP-18-004-MJ-PA-T2-D10,Good
3,0/1,0/1,0/1,0/0,0/0,0/0,0/0,1/1,1/1,1/1,...,0/0,0/0,0/1,0/1,0/1,0/0,0/1,0/1,CaP-18-006-MJ-PA-T1-C4,Good
4,1/1,0/1,0/1,0/1,0/1,0/1,0/1,1/1,1/1,1/1,...,0/1,0/0,0/0,0/0,0/1,0/1,0/1,0/1,CaP-18-030-MJ-PI-T17-H3,Good


In [3]:
samples_id = data['sample_id']
samples_class = data['class']
snps_data = data.iloc[:, :-2]

## Exploratory analysis of data

We perform a quick exploratory analysis of the data. We observe the range of values and format the columns of the *dataframe*. We also do a imputation of the missing values according to the biological criterion of the study.

In [4]:
snps_data.head()

,rs2493297,rs2493292,rs2493291,rs3820586,rs3795713,rs4342887,rs1140279,rs645128,rs659659,rs2359242,...,rs7259554,rs959829,rs6060048,rs7520,rs1789953,rs1788466,rs12774,rs1047209,rs2839173,rs2839174
0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,1/1,1/1,1/1,...,0/1,0/0,0/0,1/1,0/1,0/1,1/1,0/0,0/0,0/0
1,0/1,0/0,0/0,0/1,0/1,0/1,0/1,1/1,0/1,0/1,...,0/0,0/1,0/1,1/1,0/0,0/0,0/1,0/1,0/1,0/1
2,0/0,0/0,0/0,0/0,0/0,0/0,0/0,1/1,1/1,1/1,...,0/1,0/0,0/0,0/0,0/0,0/1,0/1,0/0,0/1,0/1
3,0/1,0/1,0/1,0/0,0/0,0/0,0/0,1/1,1/1,1/1,...,1/1,0/0,0/0,0/0,0/1,0/1,0/1,0/0,0/1,0/1
4,1/1,0/1,0/1,0/1,0/1,0/1,0/1,1/1,1/1,1/1,...,1/1,0/1,0/1,0/0,0/0,0/0,0/1,0/1,0/1,0/1


In [5]:
snps_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Columns: 190 entries, rs2493297 to rs2839174
dtypes: object(190)
memory usage: 509.3+ KB


In [6]:
snps_data.describe()

,rs2493297,rs2493292,rs2493291,rs3820586,rs3795713,rs4342887,rs1140279,rs645128,rs659659,rs2359242,...,rs7259554,rs959829,rs6060048,rs7520,rs1789953,rs1788466,rs12774,rs1047209,rs2839173,rs2839174
count,343,343,343,343,343,343,343,343,343,343,...,343,343,343,343,343,343,343,343,343,343
unique,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
top,0/1,0/0,0/0,0/0,0/0,0/0,0/0,1/1,1/1,1/1,...,0/1,0/1,0/1,0/0,0/0,0/0,1/1,0/0,0/1,0/1
freq,152,218,222,240,240,244,244,239,185,185,...,172,155,166,152,241,235,161,233,177,177


### Variables formatting for the genetic algorithm

Finally, we convert all the variables to factor type, so that they can be correctly passed to the genetic algorithm.

In [8]:
snps_data = snps_data.astype('category')

In [9]:
snps_data.head()

,rs2493297,rs2493292,rs2493291,rs3820586,rs3795713,rs4342887,rs1140279,rs645128,rs659659,rs2359242,...,rs7259554,rs959829,rs6060048,rs7520,rs1789953,rs1788466,rs12774,rs1047209,rs2839173,rs2839174
0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,1/1,1/1,1/1,...,0/1,0/0,0/0,1/1,0/1,0/1,1/1,0/0,0/0,0/0
1,0/1,0/0,0/0,0/1,0/1,0/1,0/1,1/1,0/1,0/1,...,0/0,0/1,0/1,1/1,0/0,0/0,0/1,0/1,0/1,0/1
2,0/0,0/0,0/0,0/0,0/0,0/0,0/0,1/1,1/1,1/1,...,0/1,0/0,0/0,0/0,0/0,0/1,0/1,0/0,0/1,0/1
3,0/1,0/1,0/1,0/0,0/0,0/0,0/0,1/1,1/1,1/1,...,1/1,0/0,0/0,0/0,0/1,0/1,0/1,0/0,0/1,0/1
4,1/1,0/1,0/1,0/1,0/1,0/1,0/1,1/1,1/1,1/1,...,1/1,0/1,0/1,0/0,0/0,0/0,0/1,0/1,0/1,0/1


In [10]:
snps_data['rs10000037'].cat.categories

Index(['0/0', '0/1', '1/1'], dtype='object')

Pass the SNP matrix to One-Hot-Encoding format for the correct usage by the *Random Forest* algorithm of the genetic algorithm.

In [11]:
snps_data = pd.get_dummies(snps_data, drop_first=True)
snps_data.head()

,rs2493297_0/1,rs2493297_1/1,rs2493292_0/1,rs2493292_1/1,rs2493291_0/1,rs2493291_1/1,rs3820586_0/1,rs3820586_1/1,rs3795713_0/1,rs3795713_1/1,...,rs1788466_0/1,rs1788466_1/1,rs12774_0/1,rs12774_1/1,rs1047209_0/1,rs1047209_1/1,rs2839173_0/1,rs2839173_1/1,rs2839174_0/1,rs2839174_1/1
0,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
1,True,False,False,False,False,False,True,False,True,False,...,False,False,True,False,True,False,True,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,True,False,True,False,False,False,True,False,True,False
3,True,False,True,False,True,False,False,False,False,False,...,True,False,True,False,False,False,True,False,True,False
4,False,True,True,False,True,False,True,False,True,False,...,False,False,True,False,True,False,True,False,True,False


In [12]:
snps_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Columns: 378 entries, rs2493297_0/1 to rs2839174_1/1
dtypes: bool(378)
memory usage: 126.7 KB


## Genetic algorithm

A genetic algorithm (GA) is a technique of optimization inspired by the biological evolution that can be used for feature selection in data sets. Its working is based on the natural selection, recombination and mutation principles to find the best subset of features that maximizes a specific criterion, such as the accuracy of a supervised learning model trained with the selected features.

The general process of a GA for feature selection is as follows:

1.	Initialization: A population of candidate solutions (subsets of features) is generated, represented typically as binary chromosomes, where each gene indicates whether a feature is selected (1) or not (0).

2.  Evaluation: The quality of each solution is measured using a fitness function, which can be the accuracy of a supervised learning model trained with the selected features.

3.	Selection: The best individuals of the population are selected based on their fitness, to contribute to the next generation.

4.	Crossover: Features from pairs of selected individuals are combined to generate new chromosomes (subsets of features).

5.	Mutation: Small random modifications are applied to the chromosomes to maintain diversity in the population and avoid local convergence.

6.	Replacement: A new generation of solutions is formed from the selected individuals and the generated offspring.

7.	Iteration: The steps are repeated until a stopping criterion is met, such as a maximum number of generations or convergence of the algorithm.

This approach allows efficient exploration of large search spaces to find optimal subsets of features that improve the model's performance while reducing the dimensionality of the dataset.

### Parameters

In [33]:
# Parameters for the Genetic Algorithm
population_size = 300
generations = 1100
crossover_rate = 0.8
mutation_rate = 0.05
initial_probability = 0.7

# Fitness function to choose
fitness_algorithm = "svm"

# Parameters of Random Forest for fitness function
n_trees = 50

# Parameters for the multiprocessing
n_clusters = 4

### Split data by columns

Divide the dataset by columns according to the number of clusters available or that are to be used. Also define the number of processors to be used.

In [34]:
dfs_splitted, n_clusters = splitDataByColumn(snps_data, n_clusters)

In [35]:
dfs_splitted[0]

,rs2493297_0/1,rs2493297_1/1,rs2493292_0/1,rs2493292_1/1,rs2493291_0/1,rs2493291_1/1,rs3820586_0/1,rs3820586_1/1,rs3795713_0/1,rs3795713_1/1,...,rs2070127_1/1,rs3768889_0/1,rs3768889_1/1,rs3738951_0/1,rs3738951_1/1,rs1027340_0/1,rs1027340_1/1,rs11686067_0/1,rs11686067_1/1,rs11555646_0/1
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,True,False,True,False,...,False,True,False,True,False,True,False,True,False,True
2,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,True,True,False,True
3,True,False,True,False,True,False,False,False,False,False,...,False,True,False,True,False,True,False,True,False,False
4,False,True,True,False,True,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,True,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,False,False,False
339,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,True,True,False,True
340,True,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,False,False,True
341,True,False,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,True


In [36]:
algorithm = GeneticAlgorithm(population_size, generations, crossover_rate, mutation_rate)

In [37]:
algorithm_function = partial(algorithm.run, target=samples_class, n_trees=n_trees, algorithm=fitness_algorithm, probability=[initial_probability, 1 - initial_probability])

In [38]:
start = time.time()
with mp.Pool(n_clusters) as pool:
    results = pool.map(algorithm_function, dfs_splitted)
finish = time.time()
print(f"Tiempo de duración: {finish - start} seconds")

Tiempo de duración: 1582.0314888954163 seconds


Group the boolean results for the selected variables into a single list.

In [39]:
selected_variables = []
for result in results:
    selected_variables.extend(result[0])
print(selected_variables)

[np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1)

Show the score for the best individuals selected by each processor core.

In [28]:
for result in results:
    print(result[1])

0.5533980582524272
0.5631067961165048
0.5436893203883495
0.5533980582524272


Once executed the genetic algorithm with sufficient execution time and obtained results that we are satisfied with, we keep the features selected by the algorithm and store them in a `.tsv` file.

In [29]:
selected_variables = [True if feature == 1 else False for feature in selected_variables]
snps_data_selected = snps_data.iloc[:, selected_variables]

In [30]:
snps_data_selected["samples_id"] = samples_id
snps_data_selected["class"] = samples_class

/var/folders/lp/8xhvbnh90j7366cnys2gjhq00000gn/T/ipykernel_7134/1116814529.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  snps_data_selected["samples_id"] = samples_id
/var/folders/lp/8xhvbnh90j7366cnys2gjhq00000gn/T/ipykernel_7134/1116814529.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snps_data_selected["samples_id"] = samples_id
/var/folders/lp/8xhvbnh90j7366cnys2gjhq00000gn/T/ipykernel_7134/1116814529.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

In [31]:
snps_data_selected.head()

,rs2493297_0/1,rs2493292_1/1,rs2493291_0/1,rs2493291_1/1,rs3820586_1/1,rs3795713_1/1,rs4342887_1/1,rs1140279_0/1,rs645128_0/1,rs645128_1/1,...,rs6060048_0/1,rs7520_0/1,rs7520_1/1,rs1789953_1/1,rs1788466_1/1,rs12774_0/1,rs12774_1/1,rs1047209_0/1,samples_id,class
0,False,False,False,False,False,False,False,False,False,True,...,False,False,True,False,False,False,True,False,CaP-18-001-MJ-PI-T17-A3,Good
1,True,False,False,False,False,False,False,True,False,True,...,True,False,True,False,False,True,False,True,CaP-18-002-MJ-PA-T2-A9,Good
2,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,CaP-18-004-MJ-PA-T2-D10,Good
3,True,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,CaP-18-006-MJ-PA-T1-C4,Good
4,False,False,True,False,False,False,False,True,False,True,...,True,False,False,False,False,True,False,True,CaP-18-030-MJ-PI-T17-H3,Good


In [32]:
snps_data_selected.to_csv('data_feature_selection_GA_parellel_na_imputation_svm.tsv', sep='\t', index=False)